<a href="https://colab.research.google.com/github/amazingbutuseless/wxnwxxmxngx/blob/main/%EB%B3%B4%EC%9D%B4%EC%8A%A4%EC%98%A8%EB%A6%AC_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%ED%95%98%EA%B3%A0_%EC%8B%B6%EC%96%B4%EC%9A%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# 필요한 라이브러리 설치
!pip install nltk konlpy pandas scikit-learn matplotlib wordcloud networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 28.1 MB/s eta 0:00:00


In [36]:
# 라이브러리 임포트
import re
import pandas as pd
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from google.colab import files
import os
import warnings
warnings.filterwarnings('ignore')

# KoNLPy 임포트
from konlpy.tag import Okt, Mecab
print("KoNLPy 임포트 성공")

# KoNLPy 형태소 분석기 초기화
try:
    # Mecab이 설치되어 있으면 사용 (더 빠름)
    mecab = Mecab()
    tokenizer = mecab
    print("Mecab 형태소 분석기를 사용합니다.")
except Exception:
    # 실패 시 Okt 사용 (기본)
    okt = Okt()
    tokenizer = okt
    print("Okt 형태소 분석기를 사용합니다.")

KoNLPy 임포트 성공
Okt 형태소 분석기를 사용합니다.


In [122]:
# 한국어 불용어 목록 (조사, 접속사, 일부 대명사 등)

stopwords_df = pd.read_csv('/content/korean_stopwords.csv', encoding='utf-8')
korean_stopwords = set(stopwords_df['stopwords'].tolist())

# 자막 특화 추가 불용어
subtitle_stopwords = {
    '네', '예', '음', '어', '그래', '그렇군요', '네네', '으음', '으응', '응응', '어어',
    '아아', '그러게', '그러네', '그쵸', '안녕', '안녕하세요', '잘가요', '잘가', '감사합니다',
    '감사', '고마워', '네', '응', '어', '아니', '그래', '글쎄', '아휴', '아이고', '아이구',
    '흠', '저', '다', '이런', '그런', '음', '하하', '호호', '아하', '우와', '우아', '와', '워',
    '뭐', '왜', '어떻게', '무슨', '어디', '언제', '이제', '좀', '많이', '매우', '너무', '진짜',
    '정말', '거의', '모두', '다', '약간', '조금', '거',
    '같아요', '같아', '같은데', '같아서',
    '그게', '이번', '하이', '반갑습니다', '반갑고',
    '해야', '했고', '하면서', '되면', '있으면', '하겠습니다',
    '계신가요', '주면', '될까', '아닙니다', '해보네요',
    '제', '사실', '있으신', '하여튼', '해야지', '원래', '그래가지고', '있어가지고요', '그러면은', '어쨌든',
    '하지만', '하겠어요', '걸', '갑자기', '그때', '누가', '있어요', '여기', '아니면','곧', '잠깐',
    '있나요', '막', '그렇죠', '아까', '오케이', '했어', '하느라', '였나', '했어도',
    '이었어요', '꼭', '있는', '겠다', '계셨나요', '아니어가지고', '와중에', '했네요','않겠다','왔던', '와중', '하고', '그냥', '였지',
    '입니다', '아주', '게', '고요', '뭔가', '하는', '같은데', '있어', '됩니다', '있었어', '있어서',
    '거기', '그거', '있습니다', '없어', '난', '같아', '그것'
}

# 모든 불용어 합치기
all_stopwords = korean_stopwords.union(subtitle_stopwords)
print(f"정의된 불용어 수: {len(all_stopwords)}")

정의된 불용어 수: 701


In [123]:
# 형태소 분석 및 불용어 제거 함수 (복합 키워드 추출 기능 추가)
def tokenize_and_filter(text, extract_compound=True):
    """KoNLPy를 이용한 형태소 분석 및 불용어 제거"""

    # 1. 단일 토큰 추출
    if isinstance(tokenizer, Okt):
        # Okt 형태소 분석기 사용
        pos_tagged = tokenizer.pos(text)
    else:
        # Mecab 형태소 분석기 사용
        pos_tagged = tokenizer.pos(text)

    # 단일 토큰 필터링 (명사, 형용사, 동사만 추출)
    single_tokens = []
    for word, pos in pos_tagged:
        # 명사, 형용사, 동사 필터링 (형태소 분석기에 따라 태그가 다름)
        if isinstance(tokenizer, Okt):
            is_valid = pos in ['Noun', 'Adjective', 'Verb'] and len(word) > 1 and word not in all_stopwords
        else:  # Mecab
            is_valid = (pos.startswith('NN') or pos.startswith('VA') or pos.startswith('VV')) and len(word) > 1 and word not in all_stopwords

        if is_valid:
            single_tokens.append((word, pos))

    # 복합 키워드 추출이 필요 없으면 단일 토큰만 반환
    if not extract_compound:
        return [word for word, _ in single_tokens]

    # 2. 복합 키워드 추출 (연속된 토큰에서 특정 패턴 찾기)
    compound_tokens = []

    # 원본 텍스트에서 문장 단위로 처리 (문장 경계를 넘어 복합 키워드가 형성되지 않도록)
    sentences = re.split(r'[.!?]', text)

    for sentence in sentences:
        # 문장별 형태소 분석
        if isinstance(tokenizer, Okt):
            sentence_pos = tokenizer.pos(sentence)
        else:
            sentence_pos = tokenizer.pos(sentence)

        # 패턴 1: 형용사 + 명사 (예: "새로운 기술")
        for i in range(len(sentence_pos)-1):
            curr_word, curr_pos = sentence_pos[i]
            next_word, next_pos = sentence_pos[i+1]

            # Okt용 패턴
            if isinstance(tokenizer, Okt):
                # 형용사 + 명사
                if curr_pos == 'Adjective' and next_pos == 'Noun':
                    compound = curr_word + " " + next_word
                    if len(compound) > 3 and not any(word in all_stopwords for word in [curr_word, next_word]):
                        compound_tokens.append(compound)

                # 명사 + 명사
                if curr_pos == 'Noun' and next_pos == 'Noun':
                    compound = curr_word + " " + next_word
                    if len(compound) > 3 and not any(word in all_stopwords for word in [curr_word, next_word]):
                        compound_tokens.append(compound)

            # Mecab용 패턴
            else:
                # 형용사 + 명사
                if curr_pos.startswith('VA') and next_pos.startswith('NN'):
                    compound = curr_word + " " + next_word
                    if len(compound) > 3 and not any(word in all_stopwords for word in [curr_word, next_word]):
                        compound_tokens.append(compound)

                # 명사 + 명사
                if curr_pos.startswith('NN') and next_pos.startswith('NN'):
                    compound = curr_word + " " + next_word
                    if len(compound) > 3 and not any(word in all_stopwords for word in [curr_word, next_word]):
                        compound_tokens.append(compound)

        # 패턴 2: 동사 + 명사 (예: "달리는 자동차")
        # for i in range(len(sentence_pos)-1):
        #     curr_word, curr_pos = sentence_pos[i]
        #     next_word, next_pos = sentence_pos[i+1]

        #     # Okt용 패턴
        #     if isinstance(tokenizer, Okt):
        #         if curr_pos == 'Verb' and next_pos == 'Noun':
        #             compound = curr_word + " " + next_word
        #             if len(compound) > 4 and not any(word in all_stopwords for word in [curr_word, next_word]):
        #                 compound_tokens.append(compound)

        #     # Mecab용 패턴
        #     else:
        #         if curr_pos.startswith('VV') and next_pos.startswith('NN'):
        #             compound = curr_word + " " + next_word
        #             if len(compound) > 4 and not any(word in all_stopwords for word in [curr_word, next_word]):
        #                 compound_tokens.append(compound)

    # 3. 단일 토큰과 복합 토큰 결합 (단일 토큰은 단어만 추출)
    all_tokens = [word for word, _ in single_tokens] + compound_tokens

    return all_tokens

In [129]:
# 자막 파일 업로드 및 처리 코드 (이전과 동일)
# print("VTT 자막 파일을 업로드하세요:")
# uploaded = files.upload()
# file_name = list(uploaded.keys())[0]
file_name = '/content/0-156598630.vtt'

try:
    # 파일 읽기 (여러 인코딩 시도)
    for encoding in ['utf-8', 'cp949', 'euc-kr']:
        try:
            with open(file_name, 'r', encoding=encoding) as file:
                subtitle_text = file.read()
                print(f"파일을 {encoding} 인코딩으로 성공적으로 읽었습니다.")
                break
        except UnicodeDecodeError:
            continue

    # 전처리 적용
    processed_text = preprocess_vtt(subtitle_text)
    print(f"전처리된 텍스트 샘플 (처음 300자): \n{processed_text[:300]}...\n")

    # 형태소 분석 및 토큰화 (복합 키워드 포함)
    tokens = tokenize_and_filter(processed_text, extract_compound=True)

    # 단일 키워드와 복합 키워드 구분해서 출력
    single_tokens = [t for t in tokens if " " not in t]
    compound_tokens = [t for t in tokens if " " in t]

    print(f"추출된 전체 토큰 수: {len(tokens)}")
    print(f"단일 키워드 수: {len(single_tokens)}, 복합 키워드 수: {len(compound_tokens)}")
    # print(f"단일 키워드 예시: {single_tokens[:10]}")
    # print(f"복합 키워드 예시: {compound_tokens[:10]}")
    print(f"단일 키워드 예시: {single_tokens}")
    print(f"복합 키워드 예시: {compound_tokens}")

except Exception as e:
    print(f"파일 처리 중 오류: {e}")
    import traceback
    traceback.print_exc()

파일을 utf-8 인코딩으로 성공적으로 읽었습니다.
전처리된 텍스트 샘플 (처음 300자): 
안녕하세요 안녕 안녕 하이하이 다들 반갑고 다들 잘 지내고 계신가요 안녕 얼굴 보여 주면 안 될까 오늘 제가 아침에 피부과를 다녀왔어요 지금은 비출 수 있는 상태가 아닙니다 안녕 안녕 지금부터 ㅠㅠ 금지예요 반갑습니다 오랜만 이거 보이스 라이브 진짜 오랜만에 해보네요 엄청 뭐 바쁘게 살기도 했고 그다음에 바쁘게 살아 가지고 라이브 오는데 조금 쉽지 않았어요 ㅋㅋㅋ웃어야 겠다 좋아요 웃어주세요 캐럿들 잘 지내고 계셨나요 뭐 사실 투어 끝난 지 그렇게 엄청 오랜 시간이 지난 게 아니어가지고 그 와중에 저는 되게 준비도 많이 하고 여러 ...

추출된 전체 토큰 수: 750
단일 키워드 수: 635, 복합 키워드 수: 115
단일 키워드 예시: ['지내고', '얼굴', '보여', '오늘', '아침', '피부', '다녀왔어요', '지금', '비출', '상태', '지금', '금지', '보이스', '라이브', '바쁘게', '살기', '그다음', '바쁘게', '가지', '라이브', '오는데', '쉽지', '않았어요', '웃어야', '좋아요', '웃어주세요', '캐럿', '지내고', '투어', '끝난', '지난', '준비', '여러', '버버리', '런던', '보고', '오고', '많은', '많은', '중간고사', '시즌', '쉽지', '생각', '이었는데요', '갑작스럽게', '미루어', '쏟아지면서', '스포', '생각', '얘기', '노래', '녹음', '라이브', '최대한', '보고', '빠른', '시일', '하지', '않을까', '싶어요', '런던', '처음', '갔다', '처음', '갔다', '글래스턴베리', '영국', '들리긴', '구경', '했어가지고', '갔는데', '구경', '왔어요', '오늘', '올린', '사진', '며칠', '지났지만', '녹음', '일도', '하다', '시차', '적응', '해가지고', '사진', '올려야', '되는데요', '피곤하다', '

In [130]:
# TF-IDF 기반 키워드 추출 함수 (복합 키워드 처리 추가)
def extract_keywords(text, top_n=30):
    """TF-IDF와 추가 메트릭을 사용한 키워드 추출 (복합 키워드 포함)"""
    # 복합 키워드를 포함한 토큰화
    tokens = tokenize_and_filter(text, extract_compound=True)

    # 빈도수 계산
    word_freq = Counter(tokens)

    # 단어가 너무 적으면 처리 중단
    if len(word_freq) < 5:
        print("추출된 키워드가 너무 적습니다.")
        return []

    # 복합 키워드와 단일 키워드 구분
    single_keywords = [word for word in word_freq.keys() if " " not in word]
    compound_keywords = [word for word in word_freq.keys() if " " in word]

    print(f"고유 단일 키워드 수: {len(single_keywords)}")
    print(f"고유 복합 키워드 수: {len(compound_keywords)}")

    # 문장 단위로 분할하여 TF-IDF 계산
    sentences = re.split(r'[.!?]', text)
    sentences = [sent.strip() for sent in sentences if sent.strip()]

    # 각 문장을 토큰화하여 문서로 변환
    tokenized_sentences = []
    for sentence in sentences:
        # 문장별 토큰화 (복합 키워드 포함)
        sentence_tokens = tokenize_and_filter(sentence, extract_compound=True)
        # 토큰들을 공백으로 구분된 문자열로 변환
        tokenized_sentences.append(' '.join(sentence_tokens))

    # TF-IDF 적용
    try:
        # n-gram 범위를 1~2로 설정하여 단일 및 복합 키워드 모두 포착
        tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
        tfidf_matrix = tfidf_vectorizer.fit_transform(tokenized_sentences)

        feature_names = tfidf_vectorizer.get_feature_names_out()
        word_tfidf = {}

        # 모든 토큰에 대해 TF-IDF 점수 계산
        for word in word_freq.keys():
            # 공백이 있는 복합 키워드는 TF-IDF 계산에서 제외될 수 있으므로 별도 처리
            if " " in word:
                # 복합 키워드의 구성 단어들의 TF-IDF 평균 사용
                components = word.split()
                component_scores = []
                for component in components:
                    if component in feature_names:
                        word_idx = list(feature_names).index(component)
                        component_scores.append(sum(tfidf_matrix[:, word_idx].toarray().flatten()))

                # 구성 요소가 있으면 평균, 없으면 빈도 기반 점수
                if component_scores:
                    word_tfidf[word] = sum(component_scores) / len(component_scores)
                    # 복합 키워드에 가중치 부여 (20% 증가)
                    word_tfidf[word] *= 1.2
                else:
                    word_tfidf[word] = word_freq[word] / max(word_freq.values())
            else:
                # 단일 키워드는 기존 방식대로 TF-IDF 계산
                if word in feature_names:
                    word_idx = list(feature_names).index(word)
                    word_tfidf[word] = sum(tfidf_matrix[:, word_idx].toarray().flatten())
                else:
                    word_tfidf[word] = 0
    except Exception as e:
        print(f"TF-IDF 계산 중 오류: {e}")
        # 오류 발생 시 빈도수만 사용
        word_tfidf = {word: word_freq[word] / max(word_freq.values()) for word in word_freq}

    # 최종 가중치 계산 (TF-IDF + 빈도 + 단어 길이 + 복합 키워드 보너스)
    keywords = {}
    for word in word_freq.keys():
        # 기본 가중치 = TF-IDF 점수
        weight = word_tfidf.get(word, 0)

        # 빈도 가중치 추가
        freq_weight = min(word_freq[word] / max(word_freq.values()), 1.0)

        # 단어 길이 가중치
        length_weight = min(len(word) / 10, 1.0)

        # 복합 키워드 보너스 (이미 TF-IDF에서 가중치를 줬지만 추가 보너스)
        compound_bonus = 1.1 if " " in word else 1.0

        # 최종 가중치
        keywords[word] = (weight * 0.5 + freq_weight * 0.3 + length_weight * 0.2) * compound_bonus

    # 상위 키워드 선택
    top_keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)[:top_n]

    return top_keywords

In [127]:
# 통합 키워드 추출 및 랭킹 함수 (이전과 동일)
def extract_and_rank_subtitle_keywords(text, top_n=100):
    """두 가지 방법을 조합한 키워드 추출 및 랭킹"""
    # 두 가지 방법으로 키워드 추출
    tfidf_keywords = extract_keywords(text, top_n)
    print(f"TF-IDF 키워드 추출 완료: {len(tfidf_keywords)} 개 추출됨")

    textrank_keywords_list = textrank_keywords(text, top_n)
    print(f"TextRank 키워드 추출 완료: {len(textrank_keywords_list)} 개 추출됨")

    # 결과 병합 및 가중치 조정
    final_keywords = {}

    print(f"tfidf_keywords: {tfidf_keywords}")
    print(f"textrank_keywords_list: {textrank_keywords_list}")

    # TF-IDF 결과 추가 (60% 가중치)
    for word, weight in tfidf_keywords:
        final_keywords[word] = weight * 0.6

    # TextRank 결과 추가 (40% 가중치)
    for word, weight in textrank_keywords_list:
        if word in final_keywords:
            final_keywords[word] += weight * 0.4
        else:
            final_keywords[word] = weight * 0.4

    # 최종 결과 정렬
    result = sorted(final_keywords.items(), key=lambda x: x[1], reverse=True)[:top_n]

    # 데이터프레임으로 변환
    df = pd.DataFrame(result, columns=['Keyword', 'Weight'])

    # 단일/복합 키워드 구분 컬럼 추가
    df['Type'] = df['Keyword'].apply(lambda x: '복합 키워드' if ' ' in x else '단일 키워드')

    # 가중치 정규화 (0-1 사이)
    if not df.empty:
        max_weight = df['Weight'].max()
        df['Weight'] = df['Weight'] / max_weight

    return df

# 결과 시각화 및 저장 코드는 이전과 동일

In [131]:
# 메인 실행 코드
try:
    # processed_text 변수 확인
    if 'processed_text' not in globals() or not processed_text:
        raise ValueError("전처리된 텍스트를 찾을 수 없습니다. 이전 셀을 먼저 실행해주세요.")

    # 키워드 추출 실행
    print("키워드 추출을 시작합니다...")
    keywords_df = extract_and_rank_subtitle_keywords(processed_text)

    # 결과 출력
    print("\n추출된 키워드 (상위 30개):")

    # 키워드 유형별 개수
    single_count = len(keywords_df[keywords_df['Type'] == '단일 키워드'])
    compound_count = len(keywords_df[keywords_df['Type'] == '복합 키워드'])
    print(f"단일 키워드: {single_count}개, 복합 키워드: {compound_count}개")

    # 모든 키워드 출력
    print(keywords_df)

    # 결과 CSV 파일로 저장
    keywords_df.to_csv('subtitle_keywords.csv', index=False)
    print("\nCSV 파일로 결과가 저장되었습니다.")
    # files.download('subtitle_keywords.csv')

except Exception as e:
    print(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()

키워드 추출을 시작합니다...
고유 단일 키워드 수: 394
고유 복합 키워드 수: 95
TF-IDF 키워드 추출 완료: 100 개 추출됨
TextRank 키워드 추출 완료: 100 개 추출됨
tfidf_keywords: [('노래', np.float64(0.46905961780341376)), ('오늘', np.float64(0.3760939884502742)), ('휴지통 휴지통', np.float64(0.3477511213535402)), ('사진', np.float64(0.3332368455931314)), ('생각', np.float64(0.32598550195427745)), ('봄 최애', np.float64(0.2958581779527184)), ('최애 템', np.float64(0.2958581779527184)), ('아이콘 비하인드', np.float64(0.28466127496248445)), ('노래 끝', np.float64(0.2819301240719348)), ('휴지통', np.float64(0.2817811525405607)), ('음악', np.float64(0.2759583803302783)), ('캐럿', np.float64(0.268788401563422)), ('오늘 오늘', np.float64(0.2660726361829334)), ('기억 노래', np.float64(0.2660726361829334)), ('디 아이콘', np.float64(0.2658936334645388)), ('보이스 라이브', np.float64(0.26266127496248437)), ('게임', np.float64(0.26169978766856317)), ('기억', np.float64(0.26169978766856317)), ('좋은데 휴지통', np.float64(0.25801859033555646)), ('런던', np.float64(0.25452980890170684)), ('노래 녹음', np.float64(0.24714389